In [17]:
import os
import time
import gnupg
import keyring
from binance.spot import Spot
import numpy as np
import pandas as pd
from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_checker import check_env
import gymnasium as gym
from gymnasium import spaces

In [18]:
gpg = gnupg.GPG()
encrypted_file_path = os.path.expanduser('~/Repositorios/binance_trader/.env.gpg')
with open(encrypted_file_path, 'rb') as file:
    datos = gpg.decrypt_file(file, passphrase=keyring.get_password("GPG_Passphrase", "gpg_python"))
if datos.ok:
    env_vars = dict(line.decode('utf-8').split('=', 1) for line in datos.data.splitlines())
    api_key = env_vars.get('API_KEY')
    secret_key = env_vars.get('SECRET_KEY')
# Inicializa el cliente de Binance
client = Spot(api_key, secret_key)

In [19]:
# Obtén información sobre el intercambio
exchange_info = client.exchange_info()
# Filtra los pares de trading contra USDT
usdc_pairs = [symbol['symbol'] for symbol in exchange_info['symbols'] if symbol['quoteAsset'] == 'USDC']

In [67]:
def get_historical_data(symbols, interval='1d'):
    data_frames = {}
    
    for symbol in symbols:
        start_time = 0
        data = []
        
        while True:
            # Obtener datos de klines
            klines = client.klines(symbol=symbol, interval=interval, limit=1000, startTime=start_time)
            # Si no hay más datos, salir del bucle
            if not klines:
                break
            for kline in klines:
                data.append({
                    'open_time': kline[0],
                    'open': float(kline[1]),
                    'high': float(kline[2]),
                    'low': float(kline[3]),
                    'close': float(kline[4]),
                    'volume': float(kline[5]),
                    'quote_asset_volume': float(kline[7]),
                    'number_of_trades': kline[8],
                    'taker_buy_base_asset_volume': float(kline[9]),
                    'taker_buy_quote_asset_volume': float(kline[10])
                })
            # Actualizar start_time para la próxima solicitud
            start_time = klines[-1][0] + 1
        
        data_frames[symbol] = pd.DataFrame(data)
    # Crear el dataframe multinivel
    df_list = []
    for symbol, df in data_frames.items():
        df = df.set_index('open_time')
        df.index = pd.to_datetime(df.index, unit='ms')
        df.columns = pd.MultiIndex.from_product([[symbol], df.columns])
        df_list.append(df)
    final_df = pd.concat(df_list, axis=1)
    return final_df


In [68]:
df = get_historical_data(usdc_pairs)
df

BNBUSDC                                            \
                open      high       low     close      volume   
open_time                                                        
2018-12-15    6.6666    6.6666    4.0000    4.4839    8987.350   
2018-12-16    4.5179    4.6590    4.0104    4.6068   22085.570   
2018-12-17    4.6008    5.1190    4.5379    5.0147   34794.800   
2018-12-18    5.0647    5.5633    4.8965    5.5633   50047.380   
2018-12-19    5.5792    5.8700    5.3156    5.3806  102721.740   
...              ...       ...       ...       ...         ...   
2024-06-05  685.7000  716.3000  684.6000  699.0000   10005.936   
2024-06-06  698.8000  721.7000  692.3000  710.8000    7850.551   
2024-06-07  710.8000  725.0000  644.6000  683.8000    7932.080   
2024-06-08  683.7000  699.2000  677.9000  682.6000    2139.375   
2024-06-09  682.5000  684.8000  671.0000  677.6000    1099.354   

                                                                            \
           quote_asset_volume number_of_trades taker_buy_base_asset_volume   
open_time                                                                    
2018-12-15       4.010062e+04              420                    3657.470   
2018-12-16       1.006912e+05              511                    8674.290   
2018-12-17       1.660014e+05              519                   15590.790   
2018-12-18       2.626161e+05              864                   30044.650   
2018-12-19       5.659494e+05             1855                   42413.260   
...                       ...              ...                         ...   
2024-06-05       6.998705e+06            21373                    5360.767   
2024-06-06       5.535065e+06            14849                    4168.686   
2024-06-07       5.479829e+06            17000                    3445.350   
2024-06-08       1.469139e+06             7488                    1426.387   
2024-06-09       7.450279e+05             3902                     462.284   

                                          BTCUSDC  ...  \
           taker_buy_quote_asset_volume      open  ...   
open_time                                          ...   
2018-12-15                 1.626672e+04   3200.00  ...   
2018-12-16                 3.985591e+04   3183.47  ...   
2018-12-17                 7.487978e+04   3192.69  ...   
2018-12-18                 1.581369e+05   3507.62  ...   
2018-12-19                 2.359260e+05   3673.18  ...   
...                                 ...       ...  ...   
2024-06-05                 3.747211e+06  70544.37  ...   
2024-06-06                 2.942928e+06  71114.94  ...   
2024-06-07                 2.385333e+06  70771.36  ...   
2024-06-08                 9.807246e+05  69322.36  ...   
2024-06-09                 3.136257e+05  69296.08  ...   

                                NOTUSDC TNSRUSDC                          \
           taker_buy_quote_asset_volume     open    high     low   close   
open_time                                                                  
2018-12-15                          NaN      NaN     NaN     NaN     NaN   
2018-12-16                          NaN      NaN     NaN     NaN     NaN   
2018-12-17                          NaN      NaN     NaN     NaN     NaN   
2018-12-18                          NaN      NaN     NaN     NaN     NaN   
2018-12-19                          NaN      NaN     NaN     NaN     NaN   
...                                 ...      ...     ...     ...     ...   
2024-06-05                482510.553590   1.2432  1.6299  1.1837  1.3282   
2024-06-06                363355.537271   1.3204  1.4484  1.3007  1.3279   
2024-06-07                485477.527288   1.3264  1.3264  1.1952  1.1952   
2024-06-08                393695.793659   1.1806  1.1925  1.1035  1.1192   
2024-06-09                245438.430168   1.1267  1.2907  1.0251  1.1412   

                                                         \
             volume quote_asset_volume number_of_trades   

In [69]:
df.columns

MultiIndex([( 'BNBUSDC',                         'open'),
            ( 'BNBUSDC',                         'high'),
            ( 'BNBUSDC',                          'low'),
            ( 'BNBUSDC',                        'close'),
            ( 'BNBUSDC',                       'volume'),
            ( 'BNBUSDC',           'quote_asset_volume'),
            ( 'BNBUSDC',             'number_of_trades'),
            ( 'BNBUSDC',  'taker_buy_base_asset_volume'),
            ( 'BNBUSDC', 'taker_buy_quote_asset_volume'),
            ( 'BTCUSDC',                         'open'),
            ...
            ( 'NOTUSDC', 'taker_buy_quote_asset_volume'),
            ('TNSRUSDC',                         'open'),
            ('TNSRUSDC',                         'high'),
            ('TNSRUSDC',                          'low'),
            ('TNSRUSDC',                        'close'),
            ('TNSRUSDC',                       'volume'),
            ('TNSRUSDC',           'quote_asset_volume')